# Feature at Location Matching with Graphs
Script to analyze runs from `feature_matching_evaluation.py` First, run all experiments in this file to generate the data analyzed here.

Run `python ./projects/monty_runs/run.py -e supervised_pre_training_base supervised_pre_training_small_steps supervised_pre_training_on_mesh` to generate the pretrained models.

Run `python ./projects/monty_runs/run.py -e full_rotation_eval partial_rotation_eval_base partial_rotation_eval_on_mesh unseen_rotations untested_rotations sampling_learns3_infs3 sampling_learns3_infs5 sampling_learns3_infs6 sampling_learns5_infs3 sampling_learns5_infs6 translation_left_right translation_up_down translation_closer translation_further_away max_match_dist01 max_match_dist001 no_hue unscaled_curvature principal_curvatures low_res_sensor_patch` to evaluate the detection performance under different conditions.

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

from tbp.monty.frameworks.utils.logging_utils import (load_stats,
                                                        check_rotation_accuracy,
                                                        get_time_stats,
                                                        print_overall_stats,
                                                        get_overall_pose_error)
from tbp.monty.frameworks.utils.plot_utils import (
    plot_graph, 
    plot_feature_matching_animation,
    show_one_step,
    plot_rotation_stat_animation,
    plot_detection_stat_animation)
from tbp.monty.frameworks.utils.logging_utils import deserialize_json_chunks

In [2]:
log_path = os.path.expanduser('~/tbp/results/monty/projects/feature_eval_runs/logs/')

pretrained_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/feature_eval_runs_xyz/")

pretrained_5S = pretrained_path + 'pretrained_ycb4_stepsize5/supervised_pre_training_base/pretrained/'
pretrained_5S_77Obj = pretrained_path + 'pretrained_ycb78_stepsize5/supervised_pre_training_all_objects/pretrained/'
pretrained_fc = pretrained_path + 'ycb_feature_change_S5_spiral/supervised_pre_training_feature_change_eval/pretrained/'
pretrained_3S = pretrained_path + 'pretrained_ycb4_stepsize3/supervised_pre_training_small_steps/pretrained/'
pretrained_rand_r_4 = pretrained_path + "pretrained_ycb4_steps5_rand_rot/supervised_pre_training_random_rot/pretrained/"
pretrained_rand_r_77 = pretrained_path + "pretrained_ycb77_steps5_rand_rot/supervised_pre_training_random_rot_all_obj/pretrained/"
pretrained_mesh = pretrained_path + 'pretrained_models_from_mesh/ycb4_3000p/'


In [ ]:
pretrained_3S_001mindist = pretrained_path + 'pretrained_ycb4_stepsize3_001mindist/supervised_pre_training_small_steps/pretrained/'

In [ ]:
_,_,_,m = load_stats(log_path + 'full_rotation_eval', load_train=False,load_eval=False,load_models=True,
                                                                load_detailed=False,
                     pretrained_dict=pretrained_3S)

In [ ]:
num_points = 0
num_obj = 0
for obj in m['pretrained'][0].keys():
    num_points = num_points + m['pretrained'][0][obj].x.shape[0]
    num_obj += 1
print(f"{num_obj} objects with an average of {np.round(num_points/num_obj)} points each.")

## Analyze Full Run in Detail

In [ ]:
_, full_rotation_eval_stats, ds, m = load_stats(log_path + 'full_rotation_eval', load_train=False,
                                                                pretrained_dict=pretrained_5S)

In [6]:
# save_name = 'full_rotation_eval_nosem'
save_name = 'full_rotation_eval_inf_pose_testing'
SAVE_FIGS = True
if SAVE_FIGS:
    save_path = './figures/' + save_name
    if not os.path.exists(save_path):
        os.mkdir(save_path)

### Overall Performance

In [ ]:
stats_to_plot = full_rotation_eval_stats
detailed_stats_to_plot = ds

In [ ]:
print_overall_stats(stats_to_plot)
pe = get_overall_pose_error(detailed_stats_to_plot)
print(f"poses error (radians): {pe}")

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
sns.histplot(data=stats_to_plot, x="performance", hue='target_object', multiple="stack", palette='pastel')
# sns.histplot(data=stats_to_plot.sort_values(by="performance"), x="performance", hue='target_object', multiple="stack")
plt.savefig(
    "./figures/"+save_name+"/feature_pred_rotated.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
stats_to_plot[stats_to_plot['performance']!='correct']

In [ ]:
plot_detection_stat_animation(detailed_stats_to_plot, 10)

### Number of Steps until Recognition
The time outs that are below max_steps (here 50) are because max steps counts all observations, also those that went off the object. Here we only show the number of matching steps which are only performed when we are on the object.

In [ ]:
plt.figure(figsize=(10,7))
stats_to_plot.loc[stats_to_plot["performance"]=="patch_off_object", "Num Steps"] = 1.0
ax = sns.barplot(x='performance', y='num_steps', hue='target_object', data=stats_to_plot, palette='pastel')
plt.xlabel('')
plt.xticks(fontsize=15)
plt.ylabel('Number of Saccades', fontsize=15)
plt.yticks(fontsize=15)
plt.savefig(
    "./figures/"+save_name+"/feature_obj_rotated.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Performance for Different Rotations

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
g = sns.histplot(data=stats_to_plot, x="performance", hue="target_rotation_euler", 
                 multiple="dodge", shrink=.8, palette='twilight')
plt.savefig(
    "./figures/"+save_name+"/feature_performance_rotated.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Runtime for each Step

In [ ]:
plt.figure()
for episode in detailed_stats_to_plot:
    times = detailed_stats_to_plot[episode]['LM_0']['relative_time']
    plt.plot(times)
plt.xlabel("steps")
plt.ylabel("time (s)")
plt.savefig(
    "./figures/"+save_name+"/runtime.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Accuracy of Rotation Estimation

In [ ]:
rotation_stats = check_rotation_accuracy(detailed_stats_to_plot, last_n_step=1)

plt.figure(figsize=(15,7))
sns.set(font_scale = 1.5)
sns.histplot(data=rotation_stats, x="performance", hue='object', multiple="stack", palette='pastel')
# sns.histplot(data=stats_to_plot.sort_values(by="performance"), x="performance", hue='target_object', multiple="stack")
plt.savefig(
    "./figures/"+save_name+"/feature_rotation_perf.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
rotation_stats['detected_rotation'][23]

### Accuracy of Rotation Estimation for each Rotation

In [ ]:
plt.figure(figsize=(17,5))
sns.set(font_scale = 1.5)
g = sns.histplot(data=rotation_stats, x="performance", hue="target_rotation", 
                 multiple="dodge", shrink=.8, palette='twilight')
plt.savefig(
    "./figures/"+save_name+"/feature_rotation_perf_detailed.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Rotation Estimates over Time

In [ ]:
plot_rotation_stat_animation(detailed_stats_to_plot, 10)

## Parameter Comparisons

In [7]:
_, partial_rotation_eval_stats, base_detailed_stats, m = load_stats(log_path + 'partial_rotation_eval_base', load_train=False,
                                                                load_detailed=True, load_models=True,pretrained_dict=pretrained_5S)
sns.set(font_scale = 1.5)

...loading and checking eval statistics...


In [ ]:
plot_feature_matching_animation(base_detailed_stats, 
                                m,  
                                episode = 0, 
                                objects=['mug'],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
print_overall_stats(partial_rotation_eval_stats)
pe = get_overall_pose_error(base_detailed_stats)
print(f"poses error (radins): {pe}")

In [ ]:
base_detailed_stats['0'].keys()

In [ ]:
_, rr_stats, _, rr_m = load_stats(log_path + 'random_same_rotations_all_obj', load_train=False,
                                load_detailed=False, load_models=True,pretrained_dict=pretrained_rand_r_77)

In [ ]:
print_overall_stats(rr_stats)
# pe = get_overall_pose_error(rrd_stats)
# print(f"poses error (radins): {pe}")

In [ ]:
rr_stats[rr_stats['performance']!='correct']

In [ ]:
det_path = log_path + 'random_same_rotations_all_obj' + '/detailed_run_stats.json'
# data = deserialize_json_chunks(json_file=det_path, start=0, stop=4)
data = deserialize_json_chunks(json_file=det_path, episodes=[44])

In [ ]:
data['44'].keys()

In [ ]:
plot_feature_matching_animation(data, 
                                rr_m,  
                                episode = 44, 
                                objects=['softball', 'lemon', 'pear', 'tennis_ball'],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
%matplotlib notebook

In [ ]:
plot_graph(rr_m['0']['bowl'])
plt.show()

In [ ]:
_, rr_stats_4, _, rr_m_4 = load_stats(log_path + 'random_same_rotations', load_train=False,
                                load_detailed=False, load_models=True,pretrained_dict=pretrained_rand_r_4)

In [ ]:
print_overall_stats(rr_stats_4)

In [ ]:
_, rrnew_stats_4, _, rr_m_4 = load_stats(log_path + 'random_new_rotations', load_train=False,
                                load_detailed=False, load_models=True,pretrained_dict=pretrained_rand_r_4)

In [ ]:
print_overall_stats(rrnew_stats_4)

### Effect of Sampling (Tested by using different action amounts)

In [ ]:
_, sampling_learnS3_infS3_stats, _, _ = load_stats(log_path + 'sampling_learnS3_infS3', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnS3_infS5_stats, s_3_5_det, s_3_5_m = load_stats(log_path + 'sampling_learnS3_infS5', load_train=False,
                                                                load_detailed=True, load_models=True,)
_, sampling_learnS3_infS5_all_rot_stats, s_3_5_ar_det, _ = load_stats(log_path + 'sampling_learnS3_infS5_all_rot', load_train=False,
                                                                load_detailed=True, load_models=False,)
_, sampling_learnS3_infS6_stats, _, _ = load_stats(log_path + 'sampling_learnS3_infS6', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnS5_infS3_stats, _, _ = load_stats(log_path + 'sampling_learnS5_infS3', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnS5_infS6_stats, _, _ = load_stats(log_path + 'sampling_learnS5_infS6', load_train=False,
                                                                load_detailed=False, load_models=False,)


In [ ]:
_, sampling_learnS3_infS5_stats, s_3_5_det, s_3_5_m = load_stats(log_path + 'sampling_learnS3_infS5', load_train=False,
                                                                load_detailed=True, load_models=True,)

In [ ]:
_, sampling_learnfc3_infs5_stats, s_fc3_5_det, s_fc3_5_m = load_stats(log_path + 'sampling_learnfc3_infs5', load_train=False,
                                                                load_detailed=True, load_models=True,)


In [ ]:
%matplotlib notebook

In [ ]:
plot_graph(s_fc3_5_m['0']['mug'])
plt.show()

In [ ]:
s_fc3_5_m['0']['mug']

In [ ]:
print_overall_stats(sampling_learnS3_infS5_stats)
print(f"poses error (radians): {get_overall_pose_error(s_3_5_det)}")

In [ ]:
print_overall_stats(sampling_learnfc3_infs5_stats)
print(f"poses error (radians): {get_overall_pose_error(s_fc3_5_det)}")

In [ ]:
_, sampling_learnS3_infS5_all_rot_stats, s_3_5_ar_det, _ = load_stats(log_path + 'sampling_learnS3_infS5_all_rot', load_train=False,
                                                                load_detailed=True, load_models=False,)

In [ ]:
print_overall_stats(sampling_learnS3_infS5_all_rot_stats)
print(f"poses error (radians): {get_overall_pose_error(s_3_5_ar_det)}")

In [ ]:
# _, sampling_3_5_no_pose_stats, _, _ = load_stats(log_path + 'sampling_3_5_no_pose', load_train=False,
#                                                                 load_detailed=False, load_models=False,)
# _, sampling_3_5_no_pose_all_rot_stats, _, _ = load_stats(log_path + 'sampling_3_5_no_pose_all_rot', load_train=False,
#                                                                 load_detailed=False, load_models=False,)
_, sampling_3_5_no_curv_dir_stats, s_3_5_nocurv_det, _ = load_stats(log_path + 'sampling_3_5_no_curv_dir', load_train=False,
                                                                load_detailed=True, load_models=False,)

In [ ]:
print_overall_stats(sampling_3_5_no_pose_stats)
# print(f"poses error (radians): {get_overall_pose_error(s_3_5_nopose_det)}")
print_overall_stats(sampling_3_5_no_pose_all_rot_stats)
# print(f"poses error (radians): {get_overall_pose_error(s_3_5_ar_nopose_det)}")
print_overall_stats(sampling_3_5_no_curv_dir_stats)
print(f"poses error (radians): {get_overall_pose_error(s_3_5_nocurv_det)}")

In [ ]:
sampling_dfs = [partial_rotation_eval_stats,
                sampling_learnS3_infS3_stats,
               sampling_learnS3_infS5_stats,
               sampling_learnS3_infS6_stats,
               sampling_learnS5_infS3_stats,
               sampling_learnS5_infS6_stats,]
sampling_results = pd.concat(sampling_dfs, keys=['5:5', '3:3', '3:5', '3:6', '5:3', '5:6'])
sampling_results.index.names = ['experiment', 'index']

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=sampling_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','springgreen','darkgreen','limegreen','teal','mediumturquoise'])
legend = g.get_legend()
legend.set_title('   Action Amount\nTraining:Inference')
plt.ylim([0,100])
plt.savefig(
    "./figures/"+save_name+"/sampling_effect.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

#### Different Parameters

In [ ]:
# _, sampling_learnS5_infS6_param_tests_stats_tol4, dp, _ = load_stats(log_path + 'sampling_learnS5_infS6_param_tests', load_train=False,
#                                                                 load_detailed=True, load_models=False,)


In [ ]:
# sampling_param_dfs = [sampling_learnS5_infS6_stats,
#                 sampling_learnS5_infS6_param_tests_stats,
#                      sampling_learnS5_infS6_param_tests_stats_md,# still no hue
#                      sampling_learnS5_infS6_param_tests_stats_nogc,# stiil no hue but md back at 005
#                       sampling_learnS5_infS6_param_tests_stats_pn5, #like previous but pn tol = 0.5
#                       sampling_learnS5_infS6_param_tests_stats_tol4,# like previous but mc tol = 4
#                      ]
# sampling_param_results = pd.concat(sampling_param_dfs, 
#                                    keys=['5:6 default', 'no hue', 'md 0.01', 'no hue and gc', 
#                                          'pn angle<0.5', 'mc tolerance=4'])
# sampling_param_results.index.names = ['experiment', 'index']

In [ ]:
# plt.figure(figsize=(10,7))
# g = sns.histplot(data=sampling_param_results, x="performance", hue="experiment", 
#                  multiple="dodge", shrink=.8, stat='percent', common_norm=False,
#                 palette=['mediumturquoise', 'darkcyan', 'blue', 'darkblue','dodgerblue', 'cornflowerblue'])
# legend = g.get_legend()
# legend.set_title('   Action Amount\nTraining:Inference')
# plt.savefig(
#     "./figures/"+save_name+"/sampling_effect_no_hue_no_gc_pn05_tol4.png", bbox_inches="tight", dpi=200
# ) if SAVE_FIGS else plt.show()

In [ ]:
# The above stats are not in experiment configs, load this file instead to plot

# sampling_param_results.to_csv('sampling56_param_result.csv')

In [ ]:
# plot_detection_stat_animation(dp, 10)

#### Sampling with Feature Change Models

In [ ]:
_, sampling_learnfc_inffc_stats, _, _ = load_stats(log_path + 'sampling_learnfc_inffc', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnfc_infs3_stats, _, _ = load_stats(log_path + 'sampling_learnfc_infs3', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnfc_infs5_stats, _, _ = load_stats(log_path + 'sampling_learnfc_infs5', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnfc3_infs5_stats, _, _ = load_stats(log_path + 'sampling_learnfc3_infs5', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learnfc_infs6_stats, _, _ = load_stats(log_path + 'sampling_learnfc_infs6', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sampling_learns5_inffc_stats, _, _ = load_stats(log_path + 'sampling_learns5_inffc', load_train=False,
                                                                load_detailed=False, load_models=False,)


In [ ]:
sampling_fc_dfs = [sampling_learnfc_inffc_stats,
                sampling_learnfc_infs3_stats,
               sampling_learnfc_infs5_stats,
                sampling_learnfc3_infs5_stats,
               sampling_learnfc_infs6_stats,
               sampling_learns5_inffc_stats,]
sampling_fc_dfs = pd.concat(sampling_fc_dfs, keys=['fc5:fc5', 'fc5:3', 'fc5:5','fc3:5', 'fc5:6', '5:fc5'])
sampling_fc_dfs.index.names = ['experiment', 'index']

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=sampling_fc_dfs, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','teal','darkblue','lightgreen','mediumturquoise','lightblue'])
legend = g.get_legend()
legend.set_title('   Action Amount\nTraining:Inference')
plt.ylim([0,100])
plt.savefig(
    "./figures/"+save_name+"/sampling_fc_effect.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
_, alternative_tolerances_stats, _, _ = load_stats(log_path + 'alternative_tolerances_default_setup', load_train=False,
                                                                load_detailed=False, load_models=False,)

In [ ]:
policy_fc_dfs = [alternative_tolerances_stats,
               sampling_learns5_inffc_stats,
                sampling_learnfc_inffc_stats,]
policy_fc_dfs = pd.concat(policy_fc_dfs, keys=['5:5','5:fc','fc:fc'])
policy_fc_dfs.index.names = ['experiment', 'index']
policy_fc_dfs.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=policy_fc_dfs, palette=['royalblue','darkblue','purple'])
legend = g.get_legend()
legend.set_title('Policy')
# legend.remove()
plt.savefig(
    "./figures/"+save_name+"/policy_num_steps.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Object Translation in Space

In [ ]:
_, translation_left_right_stats, _, _ = load_stats(log_path + 'translation_left_right', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, translation_up_down_stats, _, _ = load_stats(log_path + 'translation_up_down', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, translation_closer_stats, _, _ = load_stats(log_path + 'translation_closer', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, translation_further_away_stats, _, _ = load_stats(log_path + 'translation_further_away', load_train=False,
                                                                load_detailed=False, load_models=False,)

In [ ]:
translation_dfs = [partial_rotation_eval_stats,
                   translation_left_right_stats, 
                   translation_up_down_stats, 
                   translation_closer_stats, 
                   translation_further_away_stats]
translation_results = pd.concat(translation_dfs, 
                                keys=['no translation', 'left-right', 'up-down', 'closer','further away'])
translation_results.index.names = ['experiment', 'index']
# translation_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=translation_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','mediumpurple','violet','darkblue','lightblue'])
legend = g.get_legend()
legend.set_title('Object Position')
plt.savefig(
    "./figures/"+save_name+"/object_position.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=translation_left_right_stats, x="performance", hue="target_position", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['indigo','rebeccapurple','mediumpurple','rebeccapurple','indigo'])
legend = g.get_legend()
legend.set_title('Translation left-right')
plt.savefig(
    "./figures/"+save_name+"/object_position_left_right.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=translation_up_down_stats, x="performance", hue="target_position", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['thistle','plum','violet','plum','thistle'])
legend = g.get_legend()
legend.set_title('Translation up-down')
plt.savefig(
    "./figures/"+save_name+"/object_position_up_down.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of using Informed Poses

In [ ]:
_, non_informed_stats, detailed_stats_uniform, _ = load_stats(log_path + 'uniform_tested_rotations', load_train=False,
                                                                load_detailed=True, load_models=False,)


In [ ]:
informed_pose_dfs = [partial_rotation_eval_stats,
               non_informed_stats,]
informed_pose_results = pd.concat(informed_pose_dfs, keys=['informed', 'uniform'])
informed_pose_results.index.names = ['experiment', 'index']
informed_pose_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=informed_pose_results, palette=['royalblue','green'])
legend = g.get_legend()
legend.set_title('Tested Poses')
plt.savefig(
    "./figures/"+save_name+"/informed_pose_testing.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
all_ds = [detailed_stats_uniform, base_detailed_stats]
all_conditions = ['uniform', 'informed']
time_stats = get_time_stats(all_ds, all_conditions)

sns.set(font_scale = 1.5)
plt.figure(figsize=(10,5))
g = sns.lineplot(data=time_stats, x="step", y="time", hue="model_type",
                palette=['royalblue','green'])
legend = g.get_legend()
legend.set_title('Tested Poses')
plt.ylabel("time (s)")
plt.xlim([0,10])
plt.savefig(
    "./figures/"+save_name+"/informed_pose_testing_run_time.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Untested Rotations

In [ ]:
_, unseen_rotations_stats, detailed_stats_unseen_rot, _ = load_stats(log_path + 'unseen_rotations', load_train=False,
                                                                load_detailed=True, load_models=False,)
_, untested_rotations_stats, _, _ = load_stats(log_path + 'untested_rotations', load_train=False,
                                                                load_detailed=False, load_models=False,)


In [ ]:
untested_rotations_results = pd.concat([partial_rotation_eval_stats, 
                                        unseen_rotations_stats,
                                        untested_rotations_stats], 
                                   keys=['yes', 'yes but previously unseen', 'no (shifted by 10)'])
untested_rotations_results.index.names = ['experiment', 'index']

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=untested_rotations_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','chocolate','brown'])
legend = g.get_legend()
legend.set_title('Tested rotation in possible poses')
plt.savefig(
    "./figures/"+save_name+"/untested_rotation.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
rotation_stats = check_rotation_accuracy(detailed_stats_unseen_rot, last_n_step=1)

plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
g = sns.histplot(data=rotation_stats, x="performance", hue='target_rotation', multiple="stack", 
             palette='twilight')
plt.title('Rotation Performance for Previously Unseen Rotations')
legend = g.get_legend()
legend.set_title('Target Rotation')
plt.savefig(
    "./figures/"+save_name+"/feature_rotation_perf_unseen_rot.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

## Effect of Different Object Scales
This should not work yet

In [ ]:
_, different_scales_stats, _, _ = load_stats(log_path + 'different_scales', load_train=False,
                                                                load_detailed=False, load_models=False,)


In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=different_scales_stats, x="performance", hue="target_scale", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['lightsteelblue','royalblue','mediumblue','navy'])
legend = g.get_legend()
legend.set_title('Object Scale')
plt.savefig(
    "./figures/"+save_name+"/object_scale.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
scale2stats = different_scales_stats[different_scales_stats['target_scale']==2]
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
g = sns.histplot(data=scale2stats, x="performance", hue='detected_rotation', multiple="stack", 
             palette='twilight')
plt.title('Rotation Performance for Different Scales\nTarget Rotation = [0, 0, 0]')
legend = g.get_legend()
legend.set_title('Detected Rotation')
plt.savefig(
    "./figures/"+save_name+"/scale2_rotation_acc.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Match Distance Parameter

In [ ]:
_, max_match_dist01_stats, _, _ = load_stats(log_path + 'max_match_dist01', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, max_match_dist001_stats, _, _ = load_stats(log_path + 'max_match_dist001', load_train=False,
                                                                load_detailed=False, load_models=False,)


In [ ]:
match_dist_dfs = [max_match_dist001_stats,
                partial_rotation_eval_stats,
               max_match_dist01_stats,]
match_dist_results = pd.concat(match_dist_dfs, keys=['0.001', '0.005', '0.01'])
match_dist_results.index.names = ['experiment', 'index']
match_dist_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
g = sns.histplot(data=match_dist_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['steelblue','royalblue','lightsteelblue'])
legend = g.get_legend()
legend.set_title('max_match_distance')
plt.savefig(
    "./figures/"+save_name+"/max_match_distance.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=match_dist_results, palette=['steelblue','royalblue','lightsteelblue'])
legend = g.get_legend()
legend.set_title('max_match_distance')
plt.savefig(
    "./figures/"+save_name+"/max_match_distance_num_steps.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Using Different Features for Matching

In [ ]:
_, no_hue_stats, _, _ = load_stats(log_path + 'no_hue', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, sc_tolerance_4_2_stats, _, _ = load_stats(log_path + 'scaled_curve_low_tolerance', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, unscaled_curvature_stats, _, _ = load_stats(log_path + 'unscaled_curvature', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, principal_curvatures_stats, _, _ = load_stats(log_path + 'principal_curvatures', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, principal_curvatures_log_stats, _, _ = load_stats(log_path + 'scaled_pc_high_tol', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, point_norm_05_stats, _, _ = load_stats(log_path + 'point_norm_angle_05', load_train=False,
                                                                load_detailed=False, load_models=False,)

In [ ]:
feature_dfs = [partial_rotation_eval_stats,
                no_hue_stats,
               sc_tolerance_4_2_stats,
               unscaled_curvature_stats,
               principal_curvatures_stats,
               principal_curvatures_log_stats,
              point_norm_05_stats,]
feature_results = pd.concat(feature_dfs, 
                            keys=['default (scaled c, \ntolerance (16,8),\npn angle <1.57)', 
                                  'no hue', 'scaled gm curvature\nlow tolerance (4,2)',
                                  'unscaled gm curvature', 'principal curvature', 
                                  'log pc high tol', 'point norm angle < 0.5'])
feature_results.index.names = ['experiment', 'index']
feature_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.2)
g = sns.histplot(data=feature_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','slategrey','gold','darkorange', 'wheat', 'tan','purple'])
legend = g.get_legend()
legend.set_title('Features')
plt.savefig(
    "./figures/"+save_name+"/feature_comparison.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=feature_results, palette=['royalblue','slategrey','gold','darkorange', 
                                                'wheat', 'tan', 'purple'])
legend = g.get_legend()
legend.set_title('Features')
# legend.remove()
plt.savefig(
    "./figures/"+save_name+"/feature_comparison_num_steps.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Different Sensor Resolution

In [ ]:
_, low_res_sensor_patch_stats, _, _ = load_stats(log_path + 'low_res_sensor_patch', load_train=False,
                                                                load_detailed=False, load_models=False,)

In [ ]:
low_res_sensor_results = pd.concat([partial_rotation_eval_stats, low_res_sensor_patch_stats], 
                                   keys=['64x64', '5x5'])
low_res_sensor_results.index.names = ['experiment', 'index']

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=low_res_sensor_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','coral'])
legend = g.get_legend()
legend.set_title('Inference Sensor Resolution')
plt.savefig(
    "./figures/"+save_name+"/low_res_sensor.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of Model Build from Mesh

In [ ]:
_, partial_rotation_eval_on_mesh_stats, _, _ = load_stats(log_path + 'partial_rotation_eval_on_mesh', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, partial_rotation_eval_on_mesh_detailed_stats, _, _ = load_stats(log_path + 'partial_rotation_eval_on_mesh_detailed', load_train=False,
                                                                load_detailed=False, load_models=False,)
_, partial_rotation_eval_on_mesh_high_tol_stats, _, _ = load_stats(log_path + 'partial_rotation_eval_on_mesh_high_tol', load_train=False,
                                                                load_detailed=False, load_models=False,)

In [ ]:
mesh_vs_sensor_results = pd.concat([partial_rotation_eval_stats, 
                                    partial_rotation_eval_on_mesh_stats,
                                   partial_rotation_eval_on_mesh_detailed_stats, 
                                   partial_rotation_eval_on_mesh_high_tol_stats], 
                                   keys=['Sensor', 'Mesh - Simplified', 'Mesh - Detailed', 'Mesh - High Tol'])
mesh_vs_sensor_results.index.names = ['experiment', 'index']
mesh_vs_sensor_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=mesh_vs_sensor_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','hotpink', 'orchid', 'purple'])
legend = g.get_legend()
legend.set_title('Pretraining with')
# plt.savefig(
#     "./figures/"+save_name+"/mesh_vs_sensor.png", bbox_inches="tight", dpi=200
# ) if SAVE_FIGS else 
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=mesh_vs_sensor_results, palette=['royalblue','hotpink', 'orchid', 'purple'])
legend = g.get_legend()
legend.set_title('Pretraining with')
plt.savefig(
    "./figures/"+save_name+"/mesh_vs_sensor_num_steps.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
sns.histplot(data=partial_rotation_eval_on_mesh_stats, x="performance", hue='target_object', multiple="stack", palette='pastel')
# sns.histplot(data=stats_to_plot.sort_values(by="performance"), x="performance", hue='target_object', multiple="stack")
plt.savefig(
    "./figures/"+save_name+"/feature_pred_mesh.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Effect of More Models in Memory

In [ ]:
_, all_obj_in_memory_stats, all_obj_in_memory_detailed, _ = load_stats(log_path + 'partial_rotation_eval_all_obj_in_memory', load_train=False,
                                                                load_detailed=True, load_models=False,)

In [ ]:
obj_in_memory_results = pd.concat([partial_rotation_eval_stats, all_obj_in_memory_stats], 
                                   keys=['4', '71'])
obj_in_memory_results.index.names = ['experiment', 'index']
obj_in_memory_results.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=obj_in_memory_results, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','lightgreen'])
legend = g.get_legend()
legend.set_title('Number of Objects in memory')
plt.savefig(
    "./figures/"+save_name+"/num_objects_in_memory.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plt.figure(figsize=(10,7))
g = sns.barplot(x='performance', y='num_steps', hue='experiment', 
                 data=obj_in_memory_results, palette=['royalblue','lightgreen'])
legend = g.get_legend()
legend.set_title('Number of Objects in memory')
plt.savefig(
    "./figures/"+save_name+"/num_objects_num_steps.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
all_ds = [base_detailed_stats, all_obj_in_memory_detailed]
all_conditions = ['4', '71']
time_stats = get_time_stats(all_ds, all_conditions)

sns.set(font_scale = 1.5)
plt.figure(figsize=(10,5))
g = sns.lineplot(data=time_stats, x="step", y="time", hue="model_type",
                palette=['royalblue','lightgreen'])
legend = g.get_legend()
legend.set_title('Number of Objects in Memory')
plt.ylabel("time (s)")
plt.xlim([0,10])
plt.savefig(
    "./figures/"+save_name+"/num_obj_in_mem_run_time.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Testing More Objects

In [ ]:
_, all_obj_eval_stats, detailed_all, models_all= load_stats(log_path + 'full_rotation_eval_all_objects', load_train=False,
                                              load_detailed=False, load_models=True,
                                                                pretrained_dict=pretrained_5S_77Obj)

In [ ]:
num_points = 0
num_obj = 0
for obj in models_all['pretrained'][0].keys():
    num_points = num_points + models_all['pretrained'][0][obj].x.shape[0]
    num_obj += 1
print(f"{num_obj} objects with an average of {np.round(num_points/num_obj)} points each.")

In [ ]:
print_overall_stats(all_obj_eval_stats)

In [ ]:
all_obj_eval_stats[all_obj_eval_stats['performance']!='correct']

#### Load Detailed Stats from Specific Episode & Visualize

In [ ]:
det_path = log_path + 'full_rotation_eval_all_objects' + '/detailed_run_stats.json'
# data = deserialize_json_chunks(json_file=det_path, start=0, stop=4)
data = deserialize_json_chunks(json_file=det_path, episodes=[85])

In [ ]:
sns.set(font_scale = 1.0)

In [ ]:
plot_feature_matching_animation(data, 
                                models_all,  
                                episode = 85, 
                                objects=['knife'],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
all_obj_eval_stats[all_obj_eval_stats['target_object']=='apple']

In [ ]:
episode = 140
plt.figure()
plt.imshow(detailed_all[str(episode)]['SM_1']['raw_observations'][0]['rgba'])
target_o = detailed_all[str(episode)]['LM_0']['target']['object']
target_r = int(detailed_all[str(episode)]['LM_0']['target']['euler_rotation'][1])
plt.title(f"{target_o} - {target_r} deg")
plt.axis('off')
plt.savefig(
    "./figures/"+save_name+"/objects/"+target_o+"_"+str(target_r)+".png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
plot_feature_matching_animation(detailed_all, 
                                models_all,  
                                episode = 181, 
                                objects=['nine_hole_peg_test', 'sugar_box', 'wood_block'],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
show_one_step(detailed_all, 
                models_all,  
                episode = 102, # overall (epochs * episodes_per_epoch)
                step=183,
                object_to_inspect="apple",
                target_rotation = [0,45,0],
                show_full_path=False,
#                 show_point_normals=True,
                color_by_curvature=True,
                ax_range=0.04)

In [ ]:
plt.figure(figsize=(10,7))
sns.set(font_scale = 1.5)
g = sns.histplot(data=all_obj_eval_stats, x="performance", hue='target_object', multiple="stack", palette='pastel')
plt.legend([],[], frameon=False)
# sns.histplot(data=stats_to_plot.sort_values(by="performance"), x="performance", hue='target_object', multiple="stack")
plt.savefig(
    "./figures/"+save_name+"/feature_pred_78_obj.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
num_obj_tested = pd.concat([partial_rotation_eval_stats, all_obj_eval_stats], 
                                   keys=['4', '78'])
num_obj_tested.index.names = ['experiment', 'index']
num_obj_tested.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(10,7))
g = sns.histplot(data=num_obj_tested, x="performance", hue="experiment", 
                 multiple="dodge", shrink=.8, stat='percent', common_norm=False,
                palette=['royalblue','lightgreen'])
legend = g.get_legend()
legend.set_title('Number of Objects Tested')
plt.savefig(
    "./figures/"+save_name+"/num_objects_tested.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

## Look at Detailed Stats and Models

In [ ]:
_, _, ds5, s5_models = load_stats(log_path + 'partial_rotation_eval_base', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_5S)
_, _, ds5_71obj, s5_71obj_models = load_stats(log_path + 'partial_rotation_eval_all_obj_in_memory', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_5S_71Obj)
_, _, ds3, s3_models = load_stats(log_path + 'sampling_learnS3_infS3', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_3S)
_, _, dsfc, fc_models = load_stats(log_path + 'sampling_learnfc_inffc', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_fc)
_, _, dsm, mesh_models = load_stats(log_path + 'partial_rotation_eval_on_mesh', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_mesh)

In [ ]:
%matplotlib notebook

In [ ]:
fc_models['pretrained']['mug'].x.shape

In [ ]:
plot_graph(s5_models['pretrained']['mug'], show_edges=False)
plt.show()

In [ ]:
plot_graph(fc_models['pretrained']['mug'], show_edges=False)
plt.show()

In [ ]:
s5_71obj_models['pretrained'].keys()

### Inspect Specific Model

In [ ]:
sns.set(font_scale = 1.2)
object_name = 'potted_meat_can'
models_source = mesh_models

graph = models_source['pretrained'][object_name]
model_pos = graph.pos

fig = plt.figure(figsize=(15,5))

ax = plt.subplot(1,1,1,projection='3d')
feat_ids = graph.feature_mapping['gaussian_curvature_sc']
# feat_ids = graph.feature_mapping['mean_curvature_sc']
colors = graph.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'{object_name}')
ax_range = 0.05
ax.set_xlim(0-ax_range, 0+ax_range)
# ax.set_ylim(1.5-ax_range,1.5+ax_range)
ax.set_ylim(0-ax_range,0+ax_range)
ax.set_zlim(0-ax_range, 0+ax_range)
# plt.savefig(
#     "./figures/"+save_name+"/graph_of_"+object_name +".png", bbox_inches="tight", dpi=200
# ) if SAVE_FIGS else 
plt.show()

### Compare Models Build from Different Sources

In [ ]:
pretrained_mesh_detailed = '../feature_eval_runs/pretrained_models_from_mesh/ycb4_allp/'
_, _, dsmd, mesh_models_detailed = load_stats(log_path + 'partial_rotation_eval_on_mesh_detailed', load_train=False, load_eval=False,
                                load_detailed=True, pretrained_dict=pretrained_mesh_detailed)

In [ ]:
sns.set(font_scale = 1.2)
object_name = 'master_chef_can'
color_feature = 'gaussian_curvature_sc'
# color_feature = 'principal_curvatures_log'

graph_sensor_s5 = s5_models['pretrained'][object_name]
model_pos_sensor_s5 = graph_sensor_s5.pos
graph_sensor_s3 = s3_models['pretrained'][object_name]
model_pos_sensor_s3 = graph_sensor_s3.pos
graph_mesh = mesh_models['pretrained'][object_name]
model_pos_mesh = graph_mesh.pos
graph_mesh_detailed = mesh_models_detailed['pretrained'][object_name]
model_pos_mesh_detailed = graph_mesh_detailed.pos

fig = plt.figure(figsize=(10,10))

ax = plt.subplot(2,2,1,projection='3d')
feat_ids = s5_models['pretrained'][object_name].feature_mapping[color_feature]
colors = graph_sensor_s5.x[:,feat_ids[0]:feat_ids[0]+1]
s = ax.scatter(
    model_pos_sensor_s5[:, 0],
    model_pos_sensor_s5[:, 1],
    model_pos_sensor_s5[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'Sensor (Action Amount 5)\n{graph_sensor_s5.x.shape[0]} points')

ax = plt.subplot(2,2,2,projection='3d')
feat_ids = s3_models['pretrained'][object_name].feature_mapping[color_feature]
colors = graph_sensor_s3.x[:,feat_ids[0]:feat_ids[0]+1]
s = ax.scatter(
    model_pos_sensor_s3[:, 0],
    model_pos_sensor_s3[:, 1],
    model_pos_sensor_s3[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'Sensor (Action Amount 3)\n{graph_sensor_s3.x.shape[0]} points')

ax = plt.subplot(2,2,3,projection='3d')
feat_ids = mesh_models['pretrained'][object_name].feature_mapping[color_feature]
colors = graph_mesh.x[:,feat_ids[0]:feat_ids[0]+1]
s = ax.scatter(
    model_pos_mesh[:, 0],
    model_pos_mesh[:, 1],
    model_pos_mesh[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'Mesh\n{graph_mesh.x.shape[0]} points')

ax = plt.subplot(2,2,4,projection='3d')
feat_ids = mesh_models_detailed['pretrained'][object_name].feature_mapping[color_feature]
colors = graph_mesh_detailed.x[:,feat_ids[0]:feat_ids[0]+1]
s = ax.scatter(
    model_pos_mesh_detailed[:, 0],
    model_pos_mesh_detailed[:, 1],
    model_pos_mesh_detailed[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'Detailed Mesh\n{graph_mesh_detailed.x.shape[0]} points')

plt.suptitle('Graph Learned with', fontsize=15)
plt.savefig(
    "./figures/"+save_name+"/model_comparison_"+object_name +".png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
%matplotlib notebook

### Visualize Distances in Graph

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.set(font_scale = 1.0)
model = mesh_models
center_point_id = 100
max_dist = 0.005
object_name = 'mug'
graph = model['pretrained'][object_name]
model_pos = graph.pos


ax = plt.subplot(1,1,1,projection='3d')
center_p = model_pos[center_point_id]
for i, p in enumerate(model_pos):
    if np.linalg.norm(center_p - p) <= max_dist:
        color = 'red'
    else:
        color = 'grey'
    s = ax.scatter(
        p[0],
        p[1],
        p[2],
        s=5,
        c=color
    )
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title(f'Max Distance = {max_dist}')
plt.show()

### Look at Runtime

In [ ]:
all_ds = [ds5, ds3, dsm, dsmd]
all_conditions = ['Sensor Default', 'Sensor Detailed', 'Mesh - Simplified', 'Mesh - Detailed']
time_stats = get_time_stats(all_ds, all_conditions)

In [ ]:
sns.set(font_scale = 1.5)
plt.figure(figsize=(10,5))
g = sns.lineplot(data=time_stats, x="step", y="time", hue="model_type",
                palette=['royalblue','springgreen','hotpink', 'orchid'])
legend = g.get_legend()
legend.set_title('Model Type')
plt.ylabel("time (s)")
plt.xlim([0,10])
plt.savefig(
    "./figures/"+save_name+"/runtime_model_detail_with_kd_search.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

In [ ]:
all_ds = [ds5, ds5_71obj]
all_conditions = ['4 Objects', '71 Objects']
time_stats_num_obj = get_time_stats(all_ds, all_conditions)

In [ ]:
sns.set(font_scale = 1.5)
plt.figure(figsize=(10,5))
g = sns.lineplot(data=time_stats_num_obj, x="step", y="time", hue="model_type",
                palette=['royalblue','lightgreen'])
legend = g.get_legend()
legend.set_title('Number of Objects in Memory')
plt.ylabel("time (s)")
plt.xlim([0,10])
plt.savefig(
    "./figures/"+save_name+"/runtime_num_objects.png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

### Show Observations from Different Angles

In [ ]:
object_name = 'potted_meat_can'
plt.figure(figsize=(15,30))
row_num = 0
for i in range(32):
    plt.subplot(8,4,i+1)
    while ds[str(row_num)]['LM_0']['target']['object'] != object_name:
        row_num +=1
    plt.imshow(ds[str(row_num)]['SM_1']['raw_observations'][0]['rgba'])
    plt.title(ds[str(row_num)]['LM_0']['target']['euler_rotation'])
    plt.axis('off')
    row_num +=1
plt.suptitle(object_name)
plt.subplots_adjust(top=0.95)
plt.savefig(
    "./figures/"+save_name+"/views_of_"+object_name+".png", bbox_inches="tight", dpi=200
) if SAVE_FIGS else plt.show()

## More Detailed Analysis of Runs

In [ ]:
plot_detection_stat_animation(ds3, 5)

In [ ]:
sns.set(font_scale = 1.0)

In [ ]:
plot_feature_matching_animation(ds5, 
                                s5_models,  
                                episode = 5, # overall (epochs * episodes_per_epoch)
                                objects=["mug", "bowl"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
plot_feature_matching_animation(ds5, 
                                s5_models,  
                                episode = 7, # overall (epochs * episodes_per_epoch)
                                objects=["mug", "bowl", "potted_meat_can", "master_chef_can"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
plot_feature_matching_animation(dsm, 
                                mesh_models,  
                                episode = 10, # overall (epochs * episodes_per_epoch)
                                objects=["potted_meat_can", "mug"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
dsm['10']['LM_0']['gaussian_curvature_sc']

In [ ]:
dsm['10']['LM_0']['principal_curvatures']

In [ ]:
plt.figure()
plt.imshow(dsm['6']['SM_1']['raw_observations'][5]['rgba'])
plt.show()

In [ ]:
idx = 5
episode = 6
obs_3d = np.array(dsm[str(episode)]['SM_0']['raw_observations'][idx]['semantic_3d'])
obs_3d[:,3] = np.linspace(1,obs_3d.shape[0],obs_3d.shape[0])
image = dsm[str(episode)]['SM_0']['raw_observations'][idx]['rgba']

obs_dim = int(np.sqrt(obs_3d.shape[0]))
half_obs_dim = obs_dim // 2
center_id = half_obs_dim + obs_dim * half_obs_dim

p = obs_3d[center_id]
pn = dsm['10']['LM_0']['point_normal'][idx]
norm_len=0.01

radius = 0.003
distances_to_point = np.linalg.norm(
    np.array(obs_3d[:,:3]) - p[:3], axis=1
)
radius_ids = np.where(distances_to_point <= radius)[0]
radius_point_cloud = obs_3d[radius_ids]
radius_point_cloud_center = np.argmin(np.linalg.norm(radius_point_cloud[:,:3]-p[:3],axis=1))

In [ ]:
from tbp.monty.frameworks.environment_utils.graph_utils import get_curvature_at_point
k1, k2, dir1, dir2 = get_curvature_at_point(
                obs_3d, center_id, pn
            )
k1r, k2r, dir1r, dir2r = get_curvature_at_point(
                radius_point_cloud, radius_point_cloud_center, pn
            )
print(f"squared - k1: {k1}, k2: {k2}")
print(f"circular - k1: {k1r}, k2: {k2r}")

In [ ]:
plt.figure(figsize=(10,5))
ax = plt.subplot(1,3,1,projection='3d')
ax.scatter(obs_3d[:,0], obs_3d[:,1], obs_3d[:,2], alpha=0.5, s=1, 
           c=obs_3d[:,3], vmin=0,vmax=max(obs_3d[:,3]))
# plt.scatter(obs_3d[center_id,0], obs_3d[center_id,1], obs_3d[center_id,2],c='red')
ax.scatter(p[0], p[1], p[2],c='red',s=10)
# plt.plot([obs_3d[center_id,0], pn[0]], [obs_3d[center_id,1],pn[1]], [obs_3d[center_id,2],pn[2]])

ax.plot([p[0], p[0] + pn[0] * norm_len],
        [p[1], p[1] + pn[1] * norm_len],
        [p[2], p[2] + pn[2] * norm_len],
        c='lightgreen',
       )
ax.plot([p[0], p[0] + dir1[0] * norm_len],
        [p[1], p[1] + dir1[1] * norm_len],
        [p[2], p[2] + dir1[2] * norm_len],
        c='red',
       )
ax.plot([p[0], p[0] + dir2[0] * norm_len],
        [p[1], p[1] + dir2[1] * norm_len],
        [p[2], p[2] + dir2[2] * norm_len],
        c='orange',
       )
ax.set_box_aspect([1,1,1])
ax.view_init(30)
plt.axis('off')
plt.title(f"k1: {np.round(k1,2)}, k2: {np.round(k2,2)}")

ax2 = plt.subplot(1,3,2,projection='3d')
ax2.scatter(radius_point_cloud[:,0], radius_point_cloud[:,1], radius_point_cloud[:,2], 
            alpha=0.5, s=1, c=radius_point_cloud[:,3], vmin=0,vmax=max(obs_3d[:,3]))
# plt.scatter(obs_3d[center_id,0], obs_3d[center_id,1], obs_3d[center_id,2],c='red')
pr = radius_point_cloud[radius_point_cloud_center]
ax2.scatter(pr[0], pr[1], pr[2],c='red',s=10)
# plt.plot([obs_3d[center_id,0], pn[0]], [obs_3d[center_id,1],pn[1]], [obs_3d[center_id,2],pn[2]])

ax2.plot([pr[0], pr[0] + pn[0] * norm_len],
        [pr[1], pr[1] + pn[1] * norm_len],
        [pr[2], pr[2] + pn[2] * norm_len],
        c='lightgreen',
       )
ax2.plot([pr[0], pr[0] + dir1r[0] * norm_len],
        [pr[1], pr[1] + dir1r[1] * norm_len],
        [pr[2], pr[2] + dir1r[2] * norm_len],
        c='red',
       )
ax2.plot([pr[0], pr[0] + dir2r[0] * norm_len],
        [pr[1], pr[1] + dir2r[1] * norm_len],
        [pr[2], pr[2] + dir2r[2] * norm_len],
        c='orange',
       )
ax2.view_init(30)
plt.title(f"k1: {np.round(k1r,2)}, k2: {np.round(k2r,2)}")
ax.set_box_aspect([1,1,1])
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
mesh_models['pretrained']['mug']['feature_mapping']

In [ ]:
plt.figure()
plt.hist(np.array(mesh_models['pretrained']['mug'].x[6]),bins=500)
plt.xlim([0,500])
plt.show()

In [ ]:
plot_feature_matching_animation(dsm, 
                                mesh_models,  
                                episode = 1, # overall (epochs * episodes_per_epoch)
                                objects=["mug", "bowl"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
plot_feature_matching_animation(dsm, 
                                mesh_models,  
                                episode = 4, # overall (epochs * episodes_per_epoch)
                                objects=["mug", "bowl"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
plot_feature_matching_animation(dsmd, 
                                mesh_models_detailed,  
                                episode = 9, # overall (epochs * episodes_per_epoch)
                                objects=["mug", "bowl"],
                                show_num_pos=None,
                                show_path=True,
                                ax_range=0.05)

In [ ]:
show_one_step(dsmd, 
                mesh_models_detailed,  
                episode = 0, # overall (epochs * episodes_per_epoch)
                step=1,
                object_to_inspect="mug",
                target_rotation = [0,0,0],
              show_full_path=True,
              color_by_curvature=True,
                ax_range=0.04)